In [1]:
import os
import sys
from railway import *

In [2]:
# Root folder directory
ROOT_DIR = "/home/marco/railway-scheduling"

# Set the current directory to root directory
os.chdir(ROOT_DIR)
sys.path.append(os.getcwd())
print(f"Current working directory: 📂 {os.getcwd()}")

Current working directory: 📂 /home/marco/railway-scheduling


In [3]:
# Define problem parameters
N = 20
T = 50
J = 40
P = 2000
K = 3

# Name of the file to load
FILENAME = f"datasets/railway_N{N}_T{T}_J{J}_P{P}_K{K}.json"

# Instantiate the Railway object
rail = Railway.load(FILENAME)
rail.model.setParam('TimeLimit', 60)

# Print the model in summary
print(rail)

Set parameter Username
Set parameter LicenseID to value 2629256
Academic license - for non-commercial use only - expires 2026-02-27
Set parameter TimeLimit to value 60
Railway scheduling problem

Parameters:
N:  20 stations
T:  50 periods
J:  40 jobs
P:  2000 passengers
K:  3 alternative routes
Aj: 40 jobs with arcs
Ja: 39 arcs with jobs
C:  0 arcs unavailable simultaneously

Optimization model:
Variables:   0
Constraints: 0
Objective:   0.0
Status:      LOADED



In [4]:
# Set constraints
rail.set_constraints()

# Set objective
rail.set_objective()

rail.model.setParam('LPWarmStart',0)
rail.model.setParam('PoolSolutions', 1)
# rail.model.params.presolve = 0
rail.model.params.cuts = 0
rail.model.params.cutpasses = 0
rail.model.params.threads = 1
rail.model.params.heuristics = 0
rail.model.params.symmetry = 0

# Solve the scheduling problem
performance = rail.optimize()

print("Performance:")
print(performance)

Set parameter LPWarmStart to value 0
Set parameter PoolSolutions to value 1
Set parameter Cuts to value 0
Set parameter CutPasses to value 0
Set parameter Threads to value 1
Set parameter Heuristics to value 0
Set parameter Symmetry to value 0
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Arch Linux")

CPU model: Intel(R) Core(TM) i7-8565U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 1 threads

Non-default parameters:
TimeLimit  60
LPWarmStart  0
Heuristics  0
Symmetry  0
Cuts  0
CutPasses  0
Threads  1
PoolSolutions  1

Optimize a model with 169330 rows, 101000 columns and 470785 nonzeros
Model fingerprint: 0x8cf3c06d
Variable types: 29500 continuous, 71500 integer (71500 binary)
Coefficient statistics:
  Matrix range     [3e-02, 3e+07]
  Objective range  [1e+03, 2e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e-01, 3e+07]
Presolve removed 151337 rows and 87290 columns
Presolve time: 0.23s

In [5]:
# Measure runtime
runtime = rail.model.Runtime
print(f"Optimization took {runtime:.4f} seconds")

Optimization took 2.1093 seconds


In [6]:
print(type(rail.S))
print(rail.S)

print()

print(type(rail.Aj))
print(rail.Aj)

print()

print(type(rail.A))
print(rail.A)

<class 'dict'>
{1: 41, 2: 18, 3: 34, 4: 12, 5: 17, 6: 13, 7: 47, 8: 34, 9: 47, 10: 36, 11: 37, 12: 3, 13: 45, 14: 36, 15: 22, 16: 11, 17: 48, 18: 37, 19: 27, 20: 12, 21: 26, 22: 3, 23: 22, 24: 45, 25: 49, 26: 38, 27: 45, 28: 47, 29: 44, 30: 42, 31: 19, 32: 29, 33: 8, 34: 25, 35: 44, 36: 31, 37: 14, 38: 8, 39: 35, 40: 24}

<class 'dict'>
{1: [(10, 20)], 2: [(14, 17)], 3: [(9, 10)], 4: [(5, 20)], 5: [(10, 15)], 6: [(8, 13)], 7: [(5, 7)], 8: [(10, 16)], 9: [(9, 15)], 10: [(10, 15)], 11: [(2, 16)], 12: [(17, 20)], 13: [(6, 15)], 14: [(3, 6)], 15: [(13, 14)], 16: [(14, 18)], 17: [(5, 15)], 18: [(3, 17)], 19: [(3, 5)], 20: [(6, 14)], 21: [(11, 20)], 22: [(13, 20)], 23: [(2, 7)], 24: [(8, 19)], 25: [(3, 10)], 26: [(11, 16)], 27: [(8, 16)], 28: [(5, 19)], 29: [(2, 10)], 30: [(8, 9)], 31: [(8, 17)], 32: [(1, 14)], 33: [(2, 6)], 34: [(11, 17)], 35: [(11, 12)], 36: [(14, 16)], 37: [(3, 13)], 38: [(17, 18)], 39: [(1, 2)], 40: [(7, 19)]}

<class 'list'>
[(1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 

In [6]:
S = rail.S.copy()
Aj_original = rail.Aj.copy()  # Keep original reference
Aj = rail.Aj.copy() 
A = rail.A.copy()
Tend = rail.Tend

# Create dictionary of named arcs
An = {}
for i, a in enumerate(A):
    An[f"a{i}"] = a

# Use a regular set instead of OrderedDict
jobs_arc_set = set()

# Convert arcs in Aj dictionary to their names
for job, job_arc_list in Aj_original.items():
    for i, arc in enumerate(job_arc_list):
        # Find the name of the arc in An
        arc_name = [name for name, value in An.items() if value == arc]
        if arc_name:
            # Replace the arc with its name
            Aj[job][i] = arc_name[0]
            
            # Add the arc to the set
            jobs_arc_set.add(arc_name[0])

# Sort the set when you need ordered output
# This sorts by the numerical portion (e.g., "a12" -> 12)
sorted_arc_names = sorted(jobs_arc_set, key=lambda x: int(x[1:]))

# Convert arcs in Aj to their position in sorted_arc_names
for job, job_arc_list in Aj.items():
    for i, arc_name in enumerate(job_arc_list):
        # Find the index of the arc name in sorted_arc_names
        index = sorted_arc_names.index(arc_name)
        # Replace the arc name with its index
        Aj[job][i] = index
        
        
    
print('sorted_arc_names')
print(sorted_arc_names)

print('Aj')
print(Aj)

print('S')
print(S)

sorted_arc_names
['a3', 'a4', 'a5', 'a8', 'a12', 'a16', 'a19', 'a33', 'a38']
Aj
{1: [2], 2: [4], 3: [7], 4: [1], 5: [0], 6: [6], 7: [8], 8: [5], 9: [2], 10: [3]}
S
{1: 3, 2: 4, 3: 1, 4: 1, 5: 7, 6: 9, 7: 5, 8: 9, 9: 1, 10: 1}


In [25]:
j0 = list(S.keys())[0]
Aj[j0]

[2]

In [33]:
import numpy as np
import plotly.graph_objects as go

fig = go.Figure()

# Collect arcs data
num_arcs = len(sorted_arc_names)
label_positions = [i + 0.5 for i in range(len(sorted_arc_names))]

# Horizontal white lines at every integer y value
for i in range(num_arcs + 1):
    fig.add_shape(type="line",
        x0=0, y0=i, x1=Tend, y1=i,
        line=dict(
            color='white',
            width=1,
        ),
    )


# Get color palette for jobs
colors = [
    "LightSkyBlue", "LightCoral", "LightGreen", "LightPink",
    "LightSalmon", "LightSteelBlue", "LightYellow", "LightGray",
    "LightGoldenRodYellow", "LightCyan"
]

# Add shapes
# j0 = list(S.keys())[0]
# fig.add_shape(type="rect",
#     x0=S[j0],
#     y0=Aj[j0][0],
#     x1=S[j0] + 1,
#     y1=Aj[j0][0] + 1,
#     line=dict(
#         color="RoyalBlue",
#         width=1,
#     ),
#     fillcolor="LightSkyBlue",
# )
# fig.update_shapes(dict(xref='x', yref='y'))

jobs = list(S.keys())
for j in jobs:
    for i in range(len(Aj[j])):
        fig.add_shape(type="rect",
            x0=S[j],
            y0=Aj[j][i],
            x1=S[j] + 1,
            y1=Aj[j][i] + 1,
            line=dict(
                color=colors[jobs.index(j) % len(colors)],
                width=1,
            ),
            fillcolor=colors[jobs.index(j) % len(colors)],
            # hovertext=f"Job {j} - Arc {sorted_arc_names[Aj[j][i]]}",
        )
        fig.update_shapes(dict(xref='x', yref='y'))


# Layout
fig.update_layout(
    width=800,
    height=600,
    xaxis=dict(
        title="Time",
        range=[0, Tend],
        tickvals=list(range(0, Tend + 1)),
    ),
    yaxis=dict(
        type="category",
        title="Arcs",
        range=[0, num_arcs],
        tickvals=label_positions,
        ticktext=sorted_arc_names,
        showgrid=False,
    ),
)


fig.show()